# Reforma

## Preâmbulo

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import time

# Track execution time                                                          
start_time = time.time()

### Parâmetros

In [2]:
# Simulation Parameters
TB_REFORMA = 'FATO_REFORMA_SAMPLE'
#TB_REFORMA = 'FATO_REFORMA'
ANO_INICIO = 1995
ANO_FIM = 2016
DADOS_FAZENDA = '../dataset/dados_fazenda.xlsx'

# Conection parameters
HOST='tama'
PORT='5432'
DBNAME='prevdb'
USER='prevdb_user'
PASS='pr3v'

### Biblioteca

In [3]:
def ds_query(sql_query):
    """
        Query Dataset

    Parâmetros
    ----------
        sql : string
            SQL query to be performed against the dataset
        
    Retorno
    -------
        Pandas Dataframe
    """
    # Connect to an existing database
    try:
        conn = psycopg2.connect("host='{}' port={} dbname='{}'user={} password={}"
                .format(HOST, PORT, DBNAME, USER, PASS))
        df = sqlio.read_sql_query(sql, conn)
        # Close communication with the database                                     
        conn.close()
        return df
    except:
        print("Unable to connect to the database")
        return

## Simulação

### FATO_REFORMA

In [4]:
sql = """
SELECT *
FROM {table_name} 
LIMIT 10
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
fato_pessoa = ds_query(sql)
fato_pessoa

,index,ano_nasc,dt_nasc,dt_obito,sexo,clientela,ano_inicio_contrib,ano_dib,idade_dib,tempo_contrib,especie,pec6_ano_dib,pec6_idade_dib,pec6_gap,pec6_prob
0,0,1935,19350106,19981105,3,1,1988,1995,60,7,41,2008.0,73.0,13,0.789282
1,1,1929,19291214,0,3,1,1989,1995,65,6,41,2009.0,80.0,15,0.631777
2,2,1939,19390913,0,3,2,1988,1995,55,7,41,2008.0,69.0,14,0.845214
3,3,1944,19441009,0,1,1,1970,1995,50,25,46,2004.0,60.0,10,0.880527
4,4,1934,19340813,20151212,3,1,1978,1995,60,17,41,1998.0,64.0,4,0.948549
5,5,1934,19341121,0,3,1,1976,1995,60,19,41,1996.0,62.0,2,0.973187
6,6,1924,19240424,0,3,2,1988,1995,70,7,41,2008.0,84.0,14,0.519235
7,7,1929,19291112,0,3,2,1970,1995,65,25,41,1995.0,66.0,1,0.979560
8,8,1940,19400114,0,3,2,1986,1995,55,9,41,2006.0,66.0,11,0.885876
9,9,1934,19340710,0,1,2,1983,1995,60,12,41,2003.0,69.0,9,0.795428


In [5]:
sql = """
SELECT 
	ESPECIE
    ,CLIENTELA
	,SEXO
    ,IDADE_DIB
	,PEC6_IDADE_DIB
	--,PEC6_TEMPO_CONTRIB
	,PEC6_GAP
	,PEC6_ANO_DIB
	,PEC6_PROB
FROM {table_name} 
WHERE PEC6_ANO_DIB = {ano}
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
fato_pessoa = ds_query(sql)
print(fato_pessoa.columns)
print(fato_pessoa.shape)

Index(['especie', 'clientela', 'sexo', 'idade_dib', 'pec6_idade_dib',
       'pec6_gap', 'pec6_ano_dib', 'pec6_prob'],
      dtype='object')
(960, 8)


In [6]:
print('ANO FIM = {}'.format(ANO_FIM))
df = fato_pessoa[['especie','clientela', 'sexo', 'idade_dib', 'pec6_idade_dib','pec6_prob','pec6_gap']]
print(df['especie'].unique())

ANO FIM = 2016
[42 41 57 46 32 92]


### Aposentados RGPS em ANO_FIM

In [7]:
df_rgps = df.pivot_table(index='idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='count').round()
df_rgps.fillna(value=0, inplace=True, downcast='infer')
df_rgps

especie   32           41               42        46 57 92   
clientela  1     2      1        2       1      2  1  1  1   
sexo       1  3  1  3   1    3   1   3   1   3  1  1  3  1  3
idade_dib                                                    
23         0  1  0  0   0    0   0   0   0   0  0  0  0  0  0
27         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
28         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
29         0  1  0  0   0    0   0   0   0   0  0  0  0  0  0
30         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
31         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
32         0  1  0  0   0    0   0   0   0   0  0  0  0  0  0
33         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
34         2  0  0  0   0    0   0   0   0   0  0  0  0  0  0
38         1  1  0  0   0    0   0   0   0   0  0  0  0  0  0
40         1  0  0  0   0    0   0   0   0   0  0  0  0  0  0
41         1  1  0  0   0    0   0   0   0   0  0  0  0  0  0
42         3  4  0  0   0    0   0   0   0   6  0  0  0  0  0
43         0  1  0  0   0    0   0   0   6   4  0  0  0  0  0
44         4  2  0  0   0    0   0   0  12   4  0  0  1  0  0
45         3  1  0  0   0    0   0   0  27   3  0  0  0  1  0
46         2  0  0  0   0    0   0   0  17   0  1  0  0  0  0
47         6  1  0  0   0    0   0   0   9   0  0  0  0  0  1
48         3  0  0  0   0    0   0   0   5  11  0  0  1  0  0
49         2  1  0  0   0    0   0   0   1   4  0  0  3  0  0
50         6  1  1  1   0    0   0   0   5   5  0  0  0  0  0
51         2  1  1  0   0    0   0   0   4   2  0  0  0  0  1
52         3  3  0  0   0    0   0   0   5   5  0  1  2  0  0
53         2  2  0  2   0    0   0   0  16   5  1  0  2  0  0
54         0  1  1  3   0    0   0   0   3   9  0  0  0  1  0
55         7  2  0  0   0    0   0  24  17   3  0  1  0  1  0
56         2  3  1  0   0    0   0  10  20   7  0  2  1  1  0
57         7  1  0  0   0    0   0   1  23   8  0  0  0  0  0
58         3  2  1  0   0    0   0   1  10   3  0  0  0  0  0
59         3  3  0  0   0    0   0   3  15   0  0  0  1  1  0
60         4  1  0  0   0  151  86   4   7   0  0  0  2  0  1
61         1  3  0  0   0   15   3   1   6   2  0  0  0  1  0
62         7  2  0  0   0   13   1   3   4   1  0  0  0  0  0
63         3  0  0  0   0   15   4   0   8   0  0  0  0  0  0
64         1  0  0  0   0    4   1   0   1   0  0  0  0  1  0
65         0  1  0  0  88    7   0   0   1   0  0  0  1  0  0
66         0  1  0  0   6    8   2   0   0   0  0  0  0  0  0
67         0  0  0  0   6    5   1   0   0   0  0  0  0  0  0
68         1  1  0  0   1    2   0   0   1   0  0  0  0  0  0
69         0  0  0  0   1    4   0   0   0   0  0  0  0  0  0
70         1  0  0  0   1    1   1   0   0   0  0  0  0  0  0
71         0  0  0  0   1    0   0   0   0   0  0  0  0  0  0
72         0  0  0  0   0    2   0   0   0   0  0  0  0  0  0
73         0  0  0  0   0    1   1   0   0   0  0  0  0  0  0
74         0  1  0  0   0    2   0   1   0   0  0  0  0  0  0
75         0  1  0  0   0    0   0   0   0   0  0  0  0  0  0
78         0  0  0  0   1    0   0   0   0   0  0  0  0  0  0

### Aposentados PEC 6/2019 em ANO_FIM

In [8]:
df_qtd = df.pivot_table(index='pec6_idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='sum').round()
df_qtd.fillna(value=0, inplace=True, downcast='infer')
#df = df.groupby(['pec6_idade_dib','sexo']).sum()
#df['qtd'] = df['pec6_prob'].apply(lambda x: round(x))
#df_qtd.to_csv('../sandbox/2016_pec_qtd_sample.csv')
df_qtd

especie        32           41               42        46  57 92   
clientela       1     2      1       2        1      2  1   1  1   
sexo            1  3  1  3   1   3   1   3    1   3  1  1   3  1  3
pec6_idade_dib                                                     
23.0            0  1  0  0   0   0   0   0    0   0  0  0   0  0  0
27.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
28.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
29.0            0  1  0  0   0   0   0   0    0   0  0  0   0  0  0
30.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
31.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
32.0            0  1  0  0   0   0   0   0    0   0  0  0   0  0  0
33.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
34.0            2  0  0  0   0   0   0   0    0   0  0  0   0  0  0
38.0            1  1  0  0   0   0   0   0    0   0  0  0   0  0  0
40.0            1  0  0  0   0   0   0   0    0   0  0  0   0  0  0
41.0            1  1  0  0   0   0   0   0    0   0  0  0   0  0  0
42.0            3  4  0  0   0   0   0   0    0   0  0  0   0  0  0
43.0            0  1  0  0   0   0   0   0    0   0  0  0   0  0  0
44.0            4  2  0  0   0   0   0   0    0   0  0  0   0  0  0
45.0            3  1  0  0   0   0   0   0    0   0  0  0   0  1  0
46.0            2  0  0  0   0   0   0   0    0   0  0  0   0  0  0
47.0            6  1  0  0   0   0   0   0    0   0  0  0   0  0  1
48.0            3  0  0  0   0   0   0   0    0   0  0  0   0  0  0
49.0            2  1  0  0   0   0   0   0    0   0  0  0   0  0  0
50.0            6  1  1  1   0   0   0   0    0   0  0  0   0  0  0
51.0            2  1  1  0   0   0   0   0    0   0  0  0   0  0  1
52.0            3  3  0  0   0   0   0   0    0   0  0  0   0  0  0
53.0            2  2  0  2   0   0   0   0    0   0  0  0   0  0  0
54.0            0  1  1  3   0   0   0   0    0   0  0  0   0  1  0
55.0            7  2  0  0   0   0   0   0    0   0  0  0   0  1  0
56.0            2  3  1  0   0   0   0   0    0   0  0  0   0  1  0
57.0            7  1  0  0   0   0   0   0    0   0  0  0   0  0  0
58.0            3  2  1  0   0   0   0   0    0   0  0  0   0  0  0
59.0            3  3  0  0   0   0   0   0    0   0  0  0   0  1  0
60.0            4  1  0  0   0   0  46  34    0   0  2  4  11  0  1
61.0            1  3  0  0   0   0  19   2    0   0  0  0   1  1  0
62.0            7  2  0  0   0  84   9   1    0  75  0  0   0  0  0
63.0            3  0  0  0   0  10   3   0    0   1  0  0   0  0  0
64.0            1  0  0  0   0  12  10   2    0   0  0  0   0  1  0
65.0            0  1  0  0  34  27   1   1  187   0  0  0   0  0  0
66.0            0  1  0  0  13  17   1   0    1   0  0  0   0  0  0
67.0            0  0  0  0  10  17   1   1    0   0  0  0   1  0  0
68.0            1  1  0  0   8   7   0   1    1   0  0  0   0  0  0
69.0            0  0  0  0  11   7   0   0    0   0  0  0   0  0  0
70.0            1  0  0  0   6  11   1   0    0   0  0  0   0  0  0
71.0            0  0  0  0   4   7   1   0    0   0  0  0   0  0  0
72.0            0  0  0  0   3   1   1   0    0   0  0  0   0  0  0
73.0            0  0  0  0   4   3   0   0    0   0  0  0   0  0  0
74.0            0  1  0  0   2   1   1   0    0   0  0  0   0  0  0
75.0            0  1  0  0   1   4   1   1    0   0  0  0   0  0  0
76.0            0  0  0  0   0   1   0   0    0   0  0  0   0  0  0
77.0            0  0  0  0   2   2   0   0    0   0  0  0   0  0  0
78.0            0  0  0  0   0   2   1   0    0   0  0  0   0  0  0
79.0            0  0  0  0   0   2   0   0    0   0  0  0   0  0  0
80.0            0  0  0  0   0   0   0   1    0   0  0  0   0  0  0
82.0            0  0  0  0   0   0   0   1    0   0  0  0   0  0  0
83.0            0  0  0  0   0   0   0   1    0   0  0  0   0  0  0
84.0            0  0  0  0   0   1   1   0    0   0  0  0   0  0  0
87.0            0  0  0  0   0   0   0   0    0   0  0  

In [9]:
df_qtd.columns

MultiIndex(levels=[[32, 41, 42, 46, 57, 92], [1, 2], [1, 3]],
           codes=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 4, 5, 5], [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1]],
           names=['especie', 'clientela', 'sexo'])

### GAP Médio PEC 6/2019 em ANO_FIM

In [10]:
df_gap = df.pivot_table(index='pec6_idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_gap', aggfunc='mean')#.round()
df_gap.fillna(value=0, inplace=True, downcast='infer')
#df_qtd.to_csv('../sandbox/2016_pec_avggap_sample.csv')
df_gap

especie        32                 41                                   \
clientela       1     2            1                     2              
sexo            1  3  1  3         1          3          1          3   
pec6_idade_dib                                                          
23.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
27.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
28.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
29.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
30.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
31.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
32.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
33.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
34.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
38.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
40.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
41.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
42.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
43.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
44.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
45.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
46.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
47.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
48.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
49.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
50.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
51.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
52.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
53.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
54.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
55.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
56.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
57.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
58.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
59.0            0  0  0  0  0.000000   0.000000   0.000000   0.000000   
60.0            0  0  0  0  0.000000   0.000000   0.000000   4.428571   
61.0            0  0  0  0  0.000000   0.000000   0.947368   5.000000   
62.0            0  0  0  0  0.000000   1.965116   2.000000   2.000000   
63.0            0  0  0  0  0.000000   2.400000   2.333333   0.000000   
64.0            0  0  0  0  0.000000   3.750000   3.636364   3.500000   
65.0            0  0  0  0  0.000000   5.000000   5.000000   9.000000   
66.0            0  0  0  0  0.923077   5.555556   6.000000   0.000000   
67.0            0  0  0  0  2.000000   6.000000   4.000000   5.000000   
68.0            0  0  0  0  2.625000   5.750000   0.000000   8.000000   
69.0            0  0  0  0  3.666667   6.500000   0.000000   0.000000   
70.0            0  0  0  0  4.714286   5.833333   4.000000   0.000000   
71.0            0  0  0  0  5.800000   6.500000   1.000000   0.000000   
72.0            0  0  0  0  6.250000   6.000000   9.000000   0.000000   
73.0            0  0  0  0  7.000000   7.500000   0.000000   0.000000   
74.0            0  0  0  0  7.500000   6.000000   1.000000   0.000000   
75.0            0  0  0  0  8.000000   7.400000   8.000000   1.000000   
76.0            0  0  0  0  0.000000   7.000000   0.000000   0.000000   
77.0            0  0  0  0  8.000000   7.666667   0.000000   0.000000   
78.0            0  0  0  0  0.000000   7.333333  17.000000   0.000000   
79.0            0  0  0  0  0.000000   9.000000   0.000000   0.000000   
80.0            0  0  0  0  0.000000   0.000000   0.000000

### Probabilidade de Sobrevivência Média PEC 6/2019 em ANO_FIM

In [11]:
df_prob = df.pivot_table(index='pec6_idade_dib', columns=['especie','clientela','sexo'], 
                    values='pec6_prob', aggfunc='mean')#.round()
df_prob.fillna(value=0, inplace=True, downcast='infer')
#df_prob.to_csv('../sandbox/2016_pec_avgprob_sample.csv')
df_prob

especie        32                 41                                      42  \
clientela       1     2            1                   2                   1   
sexo            1  3  1  3         1         3         1         3         1   
pec6_idade_dib                                                                 
23.0            0  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
27.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
28.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
29.0            0  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
30.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
31.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
32.0            0  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
33.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
34.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
38.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
40.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
41.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
42.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
43.0            0  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
44.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
45.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
46.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
47.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
48.0            1  0  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
49.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
50.0            1  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
51.0            1  1  1  0  0.000000  0.000000  0.000000  0.000000  0.000000   
52.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
53.0            1  1  0  1  0.000000  0.000000  0.000000  0.000000  0.000000   
54.0            0  1  1  1  0.000000  0.000000  0.000000  0.000000  0.000000   
55.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
56.0            1  1  1  0  0.000000  0.000000  0.000000  0.000000  0.000000   
57.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
58.0            1  1  1  0  0.000000  0.000000  0.000000  0.000000  0.000000   
59.0            1  1  0  0  0.000000  0.000000  0.000000  0.000000  0.000000   
60.0            1  1  0  0  0.000000  0.000000  1.000000  0.969992  0.000000   
61.0            1  1  0  0  0.000000  0.000000  0.984855  0.963703  0.000000   
62.0            1  1  0  0  0.000000  0.981426  0.967631  0.981067  0.000000   
63.0            1  0  0  0  0.000000  0.977203  0.962308  0.000000  0.000000   
64.0            1  0  0  0  0.000000  0.964238  0.941150  0.966976  0.000000   
65.0            0  1  0  0  1.000000  0.949549  0.916887  0.922163  0.847403   
66.0            0  1  0  0  0.978447  0.939213  0.892140  0.000000  0.976720   
67.0            0  0  0  0  0.952748  0.930183  0.917154  0.940431  0.000000   
68.0            1  1  0  0  0.938324  0.927176  0.000000  0.904483  1.000000   
69.0            0  0  0  0  0.914161  0.914877  0.000000  0.000000  0.000000   
70.0            1  0  0  0  0.884409  0.914184  0.902185  0.000000  0.000000   
71.0            0  0  0  0  0.849919  0.896425  0.964649  0.000000  0.000000   
72.0            0  0  0  0  0.828263  0.893035  0.777392  0.000000  0.000000   
73.0            0  0  0  0  0.802612  0.864717  0.000000  0.000000  0.000000   
74.0            0  1  0  0  0.781235  0.876838  0.964653  0.000000  0.000000   
75.0            0  1  0  0  0.755206  0.850555  0.755206  0.970449  0.000000   


## Considerações

* **Probabilidade de a pessoa não querer se aposentar na idade mínima**
 * Utilizar probabilidades da base?
 * Como mensurar a "vontade de trabalhar mais para aumentar o valor do benefício"?
* **Aposentadoria por idade de homens**
 * Não sofre alterações em termos de quantidade na PEC 6/2019?
* **Tempo de Contribuição**
 * É válido condisiderar que o ano de inicio de contribuição como:  `ano_inicio_contrib = ano_dib - tempo_contrib`
 * Essa abordagem desconsidera períodos de desemprego/i

## Cleanup

In [12]:
# Print out elapsed time                                                        
elapsed_time = (time.time() - start_time) / 60                                  
print("\nExecution time: {0:0.4f} minutes.".format(elapsed_time))


Execution time: 0.0052 minutes.
